# Modified version of LeNet using image augmentation and lot more layers.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
def rotate_image(image, angle = 90, scale = 1.0):
    h, w = image.shape
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, scale)
    return cv2.warpAffine(image, M, (w, h))

In [3]:
data = np.loadtxt('/kaggle/input/digit-recognizer/train.csv', delimiter = ',', skiprows = 1)

**Using rotate_image to generate more training examples**

In [4]:
y = data[:, 0]
x = data[:, 1:].reshape(-1, 28, 28)

x_train, x_cv, y_train, y_cv = train_test_split(x, y, test_size = 0.1)
x_train_temp = x_train.copy()

for angle in np.arange(-10, 15, 5):
    x_train = np.concatenate((x_train, np.array([rotate_image(image, angle, scale = 1) for image in x_train_temp])))
    
x_train = x_train.reshape(-1, 28, 28, 1)
x_cv = x_cv.reshape(-1, 28, 28, 1)
    
y_train_temp = y_train.copy()
for i in range(x_train.shape[0]//y_train_temp.shape[0] - 1):
    y_train = np.concatenate((y_train, y_train_temp))    

In [5]:
print(f'initial train shape: {x_train_temp.shape}')
print(f'new train shape: {x_train.shape}')
print(f'Got {x_train.shape[0] // x_train_temp.shape[0]} times more images')

initial train shape: (37800, 28, 28)
new train shape: (226800, 28, 28, 1)
Got 6 times more images


In [6]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout, BatchNormalization
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
YaNet = Sequential(name = 'YaNet')
YaNet.add(Conv2D(filters = 32, 
                 kernel_size = (5, 5),
                 kernel_initializer = 'he_uniform',
                 padding = 'same', 
                 activation = 'relu', 
                 input_shape = (28, 28, 1)))
YaNet.add(BatchNormalization())
YaNet.add(Conv2D(filters = 32, 
                 kernel_size = (5, 5),
                 kernel_initializer = 'he_uniform',
                 padding = 'same', 
                 activation = 'relu'))
YaNet.add(MaxPooling2D())
YaNet.add(BatchNormalization())
YaNet.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3),
                 kernel_initializer = 'he_uniform',
                 padding = 'same', 
                 activation = 'relu'))
YaNet.add(MaxPooling2D())
YaNet.add(BatchNormalization())
YaNet.add(Conv2D(filters = 128, 
                 kernel_size = (3, 3), 
                 kernel_initializer = 'he_uniform',
                 padding = 'same', 
                 activation = 'relu'))
YaNet.add(BatchNormalization())
YaNet.add(Conv2D(filters = 128, 
                 kernel_size = (3, 3), 
                 kernel_initializer = 'he_uniform',
                 padding = 'same', 
                 activation = 'relu'))
YaNet.add(MaxPooling2D())
YaNet.add(BatchNormalization())
YaNet.add(Flatten())
YaNet.add(Dense(units = 512, 
                activation = 'relu'))
YaNet.add(Dropout(0.5))
YaNet.add(BatchNormalization())
YaNet.add(Dense(units = 256, 
                activation = 'relu'))
YaNet.add(Dropout(0.5))
YaNet.add(BatchNormalization())
YaNet.add(Dense(units = 128, 
                activation = 'relu'))
YaNet.add(Dropout(0.5))
YaNet.add(BatchNormalization())
YaNet.add(Dense(units = 64, 
                activation = 'relu'))
YaNet.add(Dropout(0.5))
YaNet.add(BatchNormalization())
YaNet.add(Dense(units = 10, 
                activation = 'softmax'))

YaNet.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

YaNet.fit(x_train, y_train, 
          epochs = 100,
          validation_data = (x_cv, y_cv),
          use_multiprocessing = True)

Epoch 1/100
7088/7088 [==============================] - 46s 6ms/step - loss: 0.3121 - accuracy: 0.9143 - val_loss: 0.0478 - val_accuracy: 0.9869
Epoch 2/100
7088/7088 [==============================] - 46s 6ms/step - loss: 0.1057 - accuracy: 0.9765 - val_loss: 0.0683 - val_accuracy: 0.9826
Epoch 3/100
7088/7088 [==============================] - 45s 6ms/step - loss: 0.0713 - accuracy: 0.9847 - val_loss: 0.0504 - val_accuracy: 0.9907
Epoch 4/100
7088/7088 [==============================] - 44s 6ms/step - loss: 0.0582 - accuracy: 0.9878 - val_loss: 0.0412 - val_accuracy: 0.9924
Epoch 5/100
7088/7088 [==============================] - 45s 6ms/step - loss: 0.0472 - accuracy: 0.9904 - val_loss: 0.0421 - val_accuracy: 0.9921
Epoch 6/100
7088/7088 [==============================] - 45s 6ms/step - loss: 0.0396 - accuracy: 0.9920 - val_loss: 0.0415 - val_accuracy: 0.9924
Epoch 7/100
7088/7088 [==============================] - 47s 7ms/step - loss: 0.0364 - accuracy: 0.9927 - val_loss: 0.0577 -

In [8]:
YaNet.summary()

Model: "YaNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0     

In [9]:
x_test = np.loadtxt('/kaggle/input/digit-recognizer/test.csv', skiprows = 1, delimiter = ',')
x_test = x_test.reshape(-1, 28, 28, 1)

**Making prediction**

In [10]:
final_prediction = np.argmax(YaNet.predict(x_test), axis = 1)
output = pd.DataFrame({'ImageId': np.arange(1, x_test.shape[0]+1), 'Label': final_prediction})
output.to_csv('my_submission.csv', index = False)